In [413]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime,date
import re
import json
from pandas.api.types import is_numeric_dtype

# from simple_salesforce import Salesforce, format_soql, format_external_id
# from simple_salesforce.exceptions import SalesforceMalformedRequest
# import simpler_sf
from sqlalchemy import create_engine
#import pymysql

# from mysql import connector


# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [414]:
# usuario = 'siempreread'
# contrasena = 'S13mpr3_Ch1l3'
# url_servidor = '209.151.148.250'
puerto = "3306"
esquema = "MetLife_A"
plugin_autenticacion = "mysql_native_password"

usuario = "care_consulta"
contrasena = "CareAssistance!10"
url_servidor = "10.0.10.22"

# Conexión a la base de datos

c_conexion = "mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}?auth_plugin={5}"
c_conexion = c_conexion.format(
    usuario, contrasena, url_servidor, puerto, esquema, plugin_autenticacion
)
motor_mysql_mariadb = create_engine(c_conexion)

#table.to_sql(name='rut_unico_tiempo',con=motor_mysql_mariadb,if_exists = 'append')


# archivo de salida modificar segun archivo
# archivo = 'C:/Users/jmartinez/Desktop/casos nuevos/clientes/1/casos_faltantes_segun_archivo.xlsx'

In [415]:
# table.to_sql(name='prueba',con=motor_mysql_mariadb,if_exists = 'append')
rut_unicos = pd.DataFrame()
sql = "SELECT * FROM MetLife_A.rut_unicos WHERE LEFT(Rut_T, 1) <> '0' "
rut_unicos = pd.read_sql_query(sql, con=motor_mysql_mariadb)

rut_unicos["Rut_C"] = rut_unicos["Rut_C"].astype(str)
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace("-", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace("–", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].astype(str)
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace(".", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace(" ", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.upper()
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.strip()

rut_unicos["Rut_T"] = rut_unicos["Rut_T"].astype(str)
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace("-", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace("–", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].astype(str)
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace(".", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace(" ", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.upper()
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.strip()

rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace(",", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace(".", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("-", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("┐", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("═", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("╔", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("┌", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.title()
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.strip()

rut_unicos.shape

""" rut_unicos.to_excel(
    "F:/Empresas SF/reportes/rut_unicos_hasta_Marzo_2024.xlsx", index=False
) """

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_23396\386836739.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_23396\386836739.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_23396\386836739.py:25: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rut_unicos["Nombre"] = rut_unicos["Nombre"].str.r

' rut_unicos.to_excel(\n    "F:/Empresas SF/reportes/rut_unicos_hasta_Marzo_2024.xlsx", index=False\n) '

In [416]:
rut_unico_tiempo_ = pd.DataFrame()
sql = "SELECT Llave FROM MetLife_A.rut_unico_tiempo_2 "
rut_unico_tiempo_ = pd.read_sql_query(sql, con=motor_mysql_mariadb)

rut_unico_tiempo_.shape

(5625168, 1)

In [ ]:
""" def sacarultimo(y):
    y = y[:-1]
    return str(y)
     """

In [ ]:
""" sftp = "F:/2024/MetLife/datos_metlife.xlsx"

derty = pd.read_excel(sftp)

rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut"].apply(lambda y: sacarultimo(y))

rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].astype(str)
rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].str.upper()
rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].str.replace("-", "")
rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].str.replace("–", "")
rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].astype(str)
rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].str.replace(".", "")

rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].str.strip()

derty["rut1"] = derty["rut1"].astype(str)
derty["rut1"] = derty["rut1"].str.upper()
derty["rut1"] = derty["rut1"].str.replace("-", "")
derty["rut1"] = derty["rut1"].str.replace("–", "")
derty["rut1"] = derty["rut1"].astype(str)
derty["rut1"] = derty["rut1"].str.replace(".", "")

derty["rut1"] = derty["rut1"].str.strip() """


In [ ]:
""" rut_unicos2 = pd.merge(
    left=derty,
    right=rut_unico_tiempo_,
    how="left",
    left_on="rut1",
    right_on="rut_rut2",
)
 """

In [ ]:
""" rut_unicos2.to_excel("F:/2024/MetLife/datos_metlife_confecha.xlsx", index=False) """

In [417]:
def fechaValida(fecha):
    if (len(str(fecha)) <= 9 ):
        fecha = ""
        
    fecha = str(fecha)
    fecha1 = str(fecha)
    
    #datetime.strptime(fecha, '%Y-%m-%d')
    
    try:
        fecha = datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S')
        date_string = fecha1
        #print("date_string =", date_string)
        #print("type of date_string =", type(date_string))

        date_object = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S")

        #print("date_object =", date_object)
        #print("type of date_object =", type(date_object))
        return date_object
    except ValueError:
        return datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')

In [418]:
mes = 3
año = "2024"
fecha = año + "-" + str(mes) + "-" + "01"  
mesesDic = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
}

str(mesesDic[mes])



sftp = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/sftp.xlsx"


# nuevos = "F:/Empresas SF/reportes/carga.xlsx"


derty = pd.read_excel(sftp)

XLRDError: Can't find workbook in OLE2 compound document

In [ ]:
derty.shape

(307780, 24)

In [ ]:
metlife_sftp = pd.DataFrame()
metlife_sftp = derty.copy()

In [ ]:


metlife_sftp["rut1"] = metlife_sftp["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace("-", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace("–", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].astype(str)
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace(".", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace(" ", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.upper()
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.strip()

metlife_sftp["rut2"] = metlife_sftp["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace("-", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace("–", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].astype(str)
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace(".", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace(" ", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.upper()
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.strip()

metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace(",", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace(".", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("-", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("┐", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("═", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("╔", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("┌", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.title()
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.strip()

metlife_sftp["poliza"] = metlife_sftp["poliza"].astype(str)
metlife_sftp["poliza"] = metlife_sftp["poliza"].str.strip()



#*******************sacar primer cero
metlife_sftp.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_23396\3662042864.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_23396\3662042864.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_23396\3662042864.py:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_sftp["Nombre"] = metlife_sftp["Nomb

(307780, 24)

In [ ]:
metlife_sftp.head()

,poliza,rutE,empleador,rut2,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut1,sdad,nacimiento,sexo,...,RELACION,desde,dddd,Nombre,nombres,apellidos,NOMBRE TITULAR,email,celular,hasta
0,340006458,5387461,4,53874614,5387461.0,4,53874614,1,19949.0,F,...,CO,2021-09-01,NaT,Fernandez Ana,Ana,Fernandez,Hernandez Salazar Jorge,NaN,NaN,45535.0
1,340012327,9227659,7,92276597,9227659.0,7,92276597,1,24824.0,M,...,CO,2023-10-01,NaT,Sanhueza Avendaño Victor,Victor,Sanhueza Avendaño,Alarcon Bastias Benicia Del Carmen,beniciaalarcon@hotmail.com,976243960,45443.0
2,340002454,10257129,0,102571290,10257129.0,0,102571290,1,24820.0,M,...,CO,2021-08-01,NaT,Palma Haeble Maida,Maida,Palma Haeble,Poblete Romero Roberto Esteban,roberto.poblete@latam.com,988293037,45504.0
3,340023043,10876809,6,108768096,10876809.0,6,108768096,1,32971.0,F,...,CO,2023-01-01,NaT,Araya Tapia Yessenia,Yessenia,Araya Tapia,Lopez Cespedes Juan Fernando,JANPAO_25_@HOTMAIL.COM,950283639,45657.0
4,340001443,11347141,7,113471417,11347141.0,7,113471417,1,40675.0,F,...,HI,2023-11-01,NaT,Jaime Bozzo Paz,Paz,Jaime Bozzo,Jaime Toro Aldo Alfredo,aldojaimetoro51@gmail.com,956167532,45473.0


In [ ]:
def siPrimerCero(y):    
    if len(y) <= 4:
        y =  "NONE"
    if y[0] == "0":
        y = y[1:]
    return str(y)
    

In [ ]:
metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_sftp = metlife_sftp[(metlife_sftp["rut1"] != "NONE")]
metlife_sftp = metlife_sftp[(metlife_sftp["rut2"] != "NONE")]


metlife_sftp["Titular"] = np.where(metlife_sftp["rut1"] == metlife_sftp["rut2"],1,2 )
metlife_sftp = metlife_sftp[(metlife_sftp["Titular"] == 1)]
metlife_sftp = metlife_sftp.drop_duplicates(
    [
        "rut1","poliza",
    ],
    keep="last",
)

metlife_sftp.shape

(307780, 25)

In [ ]:
metlife_sftp.shape

(307780, 25)

In [ ]:
correo = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/correo.xlsx"
dertyc = pd.read_excel(correo)
metlife_correo = dertyc.copy()

# nuevos = "F:/Empresas SF/reportes/carga.xlsx"



metlife_correo["rut1"] = metlife_correo["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace("-", "")
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace("–", "")
metlife_correo["rut1"] = metlife_correo["rut1"].astype(str)
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace(".", "")
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace(" ", "")
metlife_correo["rut1"] = metlife_correo["rut1"].str.upper()
metlife_correo["rut1"] = metlife_correo["rut1"].str.strip()

metlife_correo["rut2"] = metlife_correo["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace("-", "")
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace("–", "")
metlife_correo["rut2"] = metlife_correo["rut2"].astype(str)
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace(".", "")
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace(" ", "")
metlife_correo["rut2"] = metlife_correo["rut2"].str.upper()
metlife_correo["rut2"] = metlife_correo["rut2"].str.strip()

metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace(",", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace(".", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("-", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("┐", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("═", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("╔", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("┌", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.title()
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.strip()

metlife_correo["poliza"] = metlife_correo["poliza"].astype(str)
metlife_correo["poliza"] = metlife_correo["poliza"].str.strip()




#*******************sacar primer cero
metlife_correo.shape
#derty = pd.read_excel(sftp)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_23396\2005110109.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_correo["rut1"] = metlife_correo["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_23396\2005110109.py:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_correo["rut2"] = metlife_correo["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_23396\2005110109.py:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_correo["Nombre"] = metlife

(75862, 19)

In [ ]:
metlife_correo["rut1"] = metlife_correo["rut1"].apply(lambda y: siPrimerCero(y))
metlife_correo["rut2"] = metlife_correo["rut2"].apply(lambda y: siPrimerCero(y))
metlife_correo = metlife_correo[(metlife_correo["rut1"] != "NONE")]
metlife_correo = metlife_correo[(metlife_correo["rut2"] != "NONE")]

metlife_correo["Titular"] = np.where(metlife_correo["rut1"] == metlife_correo["rut2"],1,2 )
metlife_correo = metlife_correo[(metlife_correo["Titular"] == 1)]
metlife_correo = metlife_correo.drop_duplicates(
    [
        "rut1","poliza"
    ],
    keep="last",
)

metlife_correo.shape

(51851, 20)

In [41]:
metlife_correo.shape

(4172, 7)

In [ ]:
web = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/web.xlsx"
dertyd = pd.read_excel(web)


In [ ]:
dertyd.head(9)

,id,ciudad,codigo_carga,codigo_postal,comuna,convenio,convenio_plan,direccion,fecha_bloqueo,fecha_modificacion,...,p_paterno,poliza,p_relacion,rut1,rut2,rsrrs,p_sexo,procesado,rut_empresa,version
0,468344,NaN,0,NaN,NaN,77,SIN PLAN,NaN,0,2024-02-23,...,GONZALEZ,340019001,1,107208984,107208984,1,1,0,96557400K,1
1,468353,NaN,0,NaN,NaN,80,SIN PLAN,NaN,0,2024-02-26,...,BAZAES,340010579,1,176364688,176364688,1,1,0,966026405,1
2,468354,NaN,0,NaN,MARCHIHUE,80,SIN PLAN,PUNTA DE MARCHIGUE SN,0,2024-02-26,...,MADARIAGA,340012071,1,187235383,187235383,1,1,0,864318002,1
3,468358,NaN,0,NaN,RANCAGUA,77,SIN PLAN,SAN PEDRO 1855 VILLA DON MATEO 4,0,2024-02-26,...,CARVAJAL,340009432,1,202107265,202107265,1,1,0,709057006,1
4,468361,QUINTERO,0,NaN,QUINTERO,80,SIN PLAN,ARTURO PRAT 2168,0,2024-02-26,...,GONZALEZ,340010579,1,76233683,76233683,1,1,0,966026405,1
5,468366,NaN,0,NaN,NaN,80,SIN PLAN,NaN,0,2024-02-26,...,ALFARO,340012071,1,135337889,135337889,1,2,0,864318002,1
6,468370,NaN,0,NaN,PUCHUNCAVÍ,80,SIN PLAN,BELLO HORIZONTE 19 LAS VENTANAS,0,2024-02-26,...,ROMERO,340010579,1,78555874,78555874,1,1,0,966026405,1
7,468374,NaN,0,NaN,PROVIDENCIA,80,SIN PLAN,EL BOSQUE 1150,0,2024-02-26,...,LACALLE,340009363,1,105793936,105793936,1,2,0,970430008,1
8,468376,NaN,0,NaN,NaN,80,SIN PLAN,NaN,0,2024-02-26,...,VERA,340010579,1,64855883,64855883,1,1,0,966026405,1


In [ ]:
metlife_web = dertyd.copy()

In [40]:
metlife_web.head()
metlife_web.shape

(4715, 40)

In [ ]:


# nuevos = "F:/Empresas SF/reportes/carga.xlsx"



metlife_web["rut1"] = metlife_web["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_web["rut1"] = metlife_web["rut1"].str.replace("-", "")
metlife_web["rut1"] = metlife_web["rut1"].str.replace("–", "")
metlife_web["rut1"] = metlife_web["rut1"].astype(str)
metlife_web["rut1"] = metlife_web["rut1"].str.replace(".", "")
metlife_web["rut1"] = metlife_web["rut1"].str.replace(" ", "")
metlife_web["rut1"] = metlife_web["rut1"].str.upper()
metlife_web["rut1"] = metlife_web["rut1"].str.strip()

metlife_web["rut2"] = metlife_web["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_web["rut2"] = metlife_web["rut2"].str.replace("-", "")
metlife_web["rut2"] = metlife_web["rut2"].str.replace("–", "")
metlife_web["rut2"] = metlife_web["rut2"].astype(str)
metlife_web["rut2"] = metlife_web["rut2"].str.replace(".", "")
metlife_web["rut2"] = metlife_web["rut2"].str.replace(" ", "")
metlife_web["rut2"] = metlife_web["rut2"].str.upper()
metlife_web["rut2"] = metlife_web["rut2"].str.strip()

metlife_web["Nombre"] = metlife_web["Nombre"].str.replace(",", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace(".", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("-", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("┐", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("═", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("╔", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("┌", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.title()
metlife_web["Nombre"] = metlife_web["Nombre"].str.strip()

metlife_web["poliza"] = metlife_web["poliza"].astype(str)
metlife_web["poliza"] = metlife_web["poliza"].str.strip()

metlife_web["rut1"] = metlife_web["rut1"].apply(lambda y: siPrimerCero(y))
metlife_web["rut2"] = metlife_web["rut2"].apply(lambda y: siPrimerCero(y))
metlife_web = metlife_web[(metlife_web["rut1"] != "NONE")]
metlife_web = metlife_web[(metlife_web["rut2"] != "NONE")]
metlife_web.shape

metlife_web["Titular"] = np.where(metlife_web["rut1"] == metlife_web["rut2"],1,2 )
metlife_web = metlife_web[(metlife_web["Titular"] == 1)]

metlife_web = metlife_web.drop_duplicates(
    [
        "rut1","poliza"
    ],
    keep="last",
)


#*******************sacar primer cero
metlife_web.shape
#derty = pd.read_excel(sftp)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_23396\2932449918.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_web["rut1"] = metlife_web["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_23396\2932449918.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_web["rut2"] = metlife_web["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_23396\2932449918.py:26: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_web["Nombre"] = metlife_web["Nombre"].

(17967, 42)

In [ ]:
webq = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/Q.xlsx"
metlife_Q = pd.read_excel(webq)

metlife_Q["rut1"] = metlife_Q["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_Q["rut1"] = metlife_Q["rut1"].str.replace("-", "")
metlife_Q["rut1"] = metlife_Q["rut1"].str.replace("–", "")
metlife_Q["rut1"] = metlife_Q["rut1"].astype(str)
metlife_Q["rut1"] = metlife_Q["rut1"].str.replace(".", "")
metlife_Q["rut1"] = metlife_Q["rut1"].str.replace(" ", "")
metlife_Q["rut1"] = metlife_Q["rut1"].str.upper()
metlife_Q["rut1"] = metlife_Q["rut1"].str.strip()

metlife_Q["rut2"] = metlife_Q["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_Q["rut2"] = metlife_Q["rut2"].str.replace("-", "")
metlife_Q["rut2"] = metlife_Q["rut2"].str.replace("–", "")
metlife_Q["rut2"] = metlife_Q["rut2"].astype(str)
metlife_Q["rut2"] = metlife_Q["rut2"].str.replace(".", "")
metlife_Q["rut2"] = metlife_Q["rut2"].str.replace(" ", "")
metlife_Q["rut2"] = metlife_Q["rut2"].str.upper()
metlife_Q["rut2"] = metlife_Q["rut2"].str.strip()

metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace(",", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace(".", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace("-", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace("┐", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace("═", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace("╔", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace("┌", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.title()
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.strip()

metlife_Q["poliza"] = metlife_Q["poliza"].astype(str)
metlife_Q["poliza"] = metlife_Q["poliza"].str.strip()

metlife_Q["rut1"] = metlife_Q["rut1"].apply(lambda y: siPrimerCero(y))
metlife_Q["rut2"] = metlife_Q["rut2"].apply(lambda y: siPrimerCero(y))
metlife_Q = metlife_Q[(metlife_Q["rut1"] != "NONE")]
metlife_Q = metlife_Q[(metlife_Q["rut2"] != "NONE")]
metlife_Q.shape

metlife_Q["Titular"] = np.where(metlife_Q["rut1"] == metlife_Q["rut2"],1,2 )
metlife_Q = metlife_Q[(metlife_Q["Titular"] == 1)]

metlife_Q = metlife_Q.drop_duplicates(
    [
        "rut1","poliza"
    ],
    keep="last",
)


#*******************sacar primer cero
metlife_Q.shape
#derty = pd.read_excel(sftp)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_23396\3947619704.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_Q["rut1"] = metlife_Q["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_23396\3947619704.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_Q["rut2"] = metlife_Q["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_23396\3947619704.py:25: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace("

(387727, 22)

In [ ]:
subir_correo = pd.DataFrame()
subir_correo["poliza_rut"] = metlife_correo["poliza"]
subir_correo["rut_rut"] = metlife_correo["rut1"]
subir_correo["nombre_rut"] = metlife_correo["Nombre"]
subir_correo["mes_rut"] = str(mes)
subir_correo["año_rut"] = str(año)
#subir_correo["fecha_rut"] = metlife_correo["fecha"].apply(lambda y: fechaValida(y))
subir_correo["Llave"] = metlife_correo["poliza"] + metlife_correo["rut1"] + str(mes) + str(año)
subir_correo["desde_rut"] = metlife_correo["desde"].apply(lambda y: fechaValida(y))
subir_correo["hasta_rut"] = metlife_correo["hasta"].apply(lambda y: fechaValida(y))
subir_correo["canal_rut"] = "email"

subir_sql = pd.DataFrame()
subir_sql["poliza_rut"] = metlife_sftp["poliza"]
subir_sql["rut_rut"] = metlife_sftp["rut1"]
subir_sql["nombre_rut"] = metlife_sftp["Nombre"]
subir_sql["mes_rut"] = str(mes)
subir_sql["año_rut"] = str(año)
#subir_sql["fecha_rut"] = metlife_sftp["fecha"].apply(lambda y: fechaValida(y))
subir_sql["Llave"] = metlife_sftp["poliza"] + metlife_sftp["rut1"] + str(mes) + str(año)
subir_sql["desde_rut"] = metlife_sftp["desde"].apply(lambda y: fechaValida(y))
subir_sql["hasta_rut"] = metlife_sftp["hasta"].apply(lambda y: fechaValida(y)) 
subir_sql["canal_rut"] = "sftp"

subir_web = pd.DataFrame()
subir_web["poliza_rut"] = metlife_web["poliza"]
subir_web["rut_rut"] = metlife_web["rut1"]
subir_web["nombre_rut"] = metlife_web["Nombre"]
subir_web["mes_rut"] = str(mes)
subir_web["año_rut"] = str(año)
#subir_web["fecha_rut"] = metlife_web["fecha"].apply(lambda y: fechaValida(y))
subir_web["Llave"] = metlife_web["poliza"] + metlife_web["rut1"] + str(mes) + str(año)
subir_web["desde_rut"] = metlife_web["desde"].apply(lambda y: fechaValida(y))
subir_web["hasta_rut"] = metlife_web["hasta"].apply(lambda y: fechaValida(y))
subir_web["canal_rut"] = "web"

subir_Q = pd.DataFrame()
subir_Q["poliza_rut"] = metlife_Q["poliza"]
subir_Q["rut_rut"] = metlife_Q["rut1"]
subir_Q["nombre_rut"] = metlife_Q["Nombre"]
subir_Q["mes_rut"] = str(mes)
subir_Q["año_rut"] = str(año)
#subir_Q["fecha_rut"] = metlife_Q["fecha"].apply(lambda y: fechaValida(y))
subir_Q["Llave"] = metlife_Q["poliza"] + metlife_web["rut1"] + str(mes) + str(año)
subir_Q["desde_rut"] = metlife_Q["desde"].apply(lambda y: fechaValida(y))
subir_Q["hasta_rut"] = metlife_Q["hasta"].apply(lambda y: fechaValida(y))
subir_Q["canal_rut"] = "Q"

mes_metlife = pd.DataFrame()
mes_metlife = pd.concat(
    [
        subir_web,
        subir_sql,
        subir_correo,
        subir_Q,
        
    ],
    axis=0,
)
mes_metlife["fecha_rut"] = pd.to_datetime(fecha, yearfirst=True)

mes_metlife = mes_metlife.drop_duplicates(
    [
        "rut_rut","poliza_rut",
    ],
    keep="last",
)
mes_metlife.head()

,poliza_rut,rut_rut,nombre_rut,mes_rut,año_rut,Llave,desde_rut,hasta_rut,canal_rut,fecha_rut
3,340009432,202107265,Eliseo Adonis Carvajal Yañez,2,2024,34000943220210726522024,2024-03-25 18:38:15,2024-03-25 18:38:16,web,2024-02-01
48,340002976,174365075,Jorge Sepulveda Nievas,2,2024,34000297617436507522024,2024-03-25 18:38:15,2024-03-25 18:38:16,web,2024-02-01
192,340001935,174956472,Miguel Angel Mendez Lobos,2,2024,34000193517495647222024,2024-03-25 18:38:15,2024-03-25 18:38:16,web,2024-02-01
193,340001935,141282115,Rodrigo Andrés Cornejo Cornejo,2,2024,34000193514128211522024,2024-03-25 18:38:15,2024-03-25 18:38:16,web,2024-02-01
195,340001935,141772872,César Antonio Diaz Hervera,2,2024,34000193514177287222024,2024-03-25 18:38:15,2024-03-25 18:38:16,web,2024-02-01


//////////////////////////////////////////////////////////////////////////

NUMERO RUTS POR POLIZA
****************************************************************

In [ ]:
mes_metlife.shape
print ( str(mes) + "-" + str(año) + " " + str(mes_metlife.shape[0]))

2-2024 396358


////////////////////////////////////////////////

NUMERO RUT UNICOS
------------------------------------------------------------------------------------------------

In [ ]:
mes_metlife2 = mes_metlife.drop_duplicates(
    [
        "rut_rut"
    ],
    keep="last",
)
mes_metlife2.shape
print (str(mes) + "-" + str(año) + " " + str(mes_metlife2.shape[0]))

2-2024 313010


In [ ]:
#subir_web["Llave"]
mes_metlife["esta"] =  mes_metlife["Llave"].isin(rut_unico_tiempo_["Llave"])
mes_metlife = mes_metlife[(mes_metlife["esta"] == False)]

mes_metlife = mes_metlife[
    [
        "poliza_rut",
        "rut_rut",
        "nombre_rut",
        "mes_rut",
        "año_rut",
        "fecha_rut",
        "Llave",
        "desde_rut",
        "hasta_rut",
        "canal_rut"
    ]
]
mes_metlife.shape

(396358, 10)

In [ ]:
mes_metlife.to_sql(
    name="rut_unico_tiempo_2",
    con=motor_mysql_mariadb,
    if_exists="append",
    index=False,
    chunksize=10000,
)

396358

In [ ]:
subir_unicos = pd.DataFrame()
subir_unicos["Rut_T"] = mes_metlife2["rut_rut"]
subir_unicos["Rut_C"] = mes_metlife2["rut_rut"]
subir_unicos["Nombre"] = mes_metlife2["nombre_rut"]
subir_unicos["Llave"] = mes_metlife2["rut_rut"] +  mes_metlife2["rut_rut"]


subir_unicos.shape

(313010, 4)

In [ ]:
subir_unicos["esta"] =  subir_unicos["Rut_T"].isin(rut_unicos["Rut_T"])
subir_unicos = subir_unicos[(subir_unicos["esta"] == False)]

subir_unicos = subir_unicos[
    [
        "Rut_T",
        "Rut_C",
        "Nombre",
        "Llave",
    ]
]
subir_unicos.shape
subir_unicos.to_sql(
    name="rut_unicos",
    con=motor_mysql_mariadb,
    if_exists="append",
    index=False,
    chunksize=10000,
)

0

In [ ]:



rut_unicos["esta"] =  rut_unicos["Rut_T"].isin(mes_metlife2["rut_rut"])
rut_unicos = rut_unicos[(rut_unicos["esta"] == False)]


In [ ]:
rut_unicos.shape

In [ ]:
rut_unicos.to_excel(
    "F:/Empresas SF/reportes/rut_unicos_No_Estan_SFTP_Agosto_2023.xlsx", index=False
)

In [ ]:
sftp = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/poliza.xlsx"
poliza = pd.read_excel(sftp)

In [ ]:
poliza.head()

In [ ]:
rut_unico_tiempo_ = pd.DataFrame()
sql = "SELECT * FROM MetLife_A.rut_unico_tiempo "
rut_unico_tiempo_ = pd.read_sql_query(sql, con=motor_mysql_mariadb)

rut_unico_tiempo_.shape

In [ ]:
rut_unicos = pd.merge(
    left=rut_unicos,
    right=rut_unico_tiempo_,
    how="left",
    left_on="Rut_T",
    right_on="rut_rut",
)

#rutesi.to_excel("F:/Empresas SF/reportes/rutsyempresa.xlsx", index=False)

In [ ]:
rut_unicos.shape

In [ ]:
rut_unicos.head()

In [ ]:
rut_unicos = rut_unicos.drop_duplicates(
    [
        "rut_rut","poliza_rut",
    ],
    keep="last",
)
rut_unicos.shape

In [ ]:
rut_unicos = pd.merge(
    left=rut_unicos,
    right=poliza,
    how="left",
    left_on="poliza_rut",
    right_on="poliza",
)

In [ ]:
rut_unicos.shape

In [ ]:
rut_unicos.head()


In [ ]:
rut_unicos.to_excel(
    "F:/Empresas SF/reportes/rut_unicos_No_Estan_SFTP_Agosto_2023_2.xlsx", index=False
)